# HAR模型 - PyTorch实现

这个notebook实现了将TensorFlow版本的HART (Human Activity Recognition Transformer) 模型迁移到PyTorch的过程。

In [ ]:
import subprocess
import os
import re
import numpy as np
import torch
import csv
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import time
import pickle
import random
import math
import logging
import shutil
import gc
import sys
import sklearn.manifold
import seaborn as sns
import argparse
import matplotlib.gridspec as gridspec
import __main__ as main
import torch.nn as nn
import torch.optim as optim

In [ ]:
# 导入自定义模块
#import model_pytorch as model
#import model_rnn as model
#import model_hybrid as model
import model_teacher as model
import utils_torch as utils

In [ ]:
'''def pick_free_gpu(threshold_mb=2000):
    """选择一个空闲内存大于阈值的GPU"""
    result = subprocess.run(
        ["nvidia-smi", "--query-gpu=memory.free", "--format=csv,nounits,noheader"],
        stdout=subprocess.PIPE,
        text=True
    )
    gpu_memory = [int(x) for x in result.stdout.strip().split('\n')]
    for idx, mem in enumerate(gpu_memory):
        if mem > threshold_mb:
            return str(idx)
    return None

# 选择GPU
gpu_id = pick_free_gpu()
if gpu_id is None:
    print("没有找到空闲GPU，使用CPU")
    device = torch.device("cpu")
else:
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu_id
    device = torch.device(f"cuda:{0}")
    print(f"使用GPU:{gpu_id}")'''

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 设置随机种子
random_seed = 1
os.environ['PYTHONHASHSEED'] = str(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
random.seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [ ]:
def is_interactive():
    """检查是否在交互式环境中运行"""
    return not hasattr(main, '__file__')

def get_layer_index_by_name(model, layername):
    """通过名称获取层的索引"""
    for name, child in model.named_modules():
        if name.endswith(layername):
            return name
    return None

## 设置超参数

In [ ]:
# 设置默认超参数
architecture = "HART"  # MobileHART, HART
dataset_name = 'UCI'  # RealWorld, HHAR, UCI, SHL, MotionSense, COMBINED
data_config = "BALANCED"  # BALANCED, UNBALANCED
show_train_verbose = 1  # 显示训练详细信息: 0, 1
segment_size = 128  # 输入窗口大小
num_input_channels = 6  # 输入通道数
learning_rate = 5e-3  # 学习率
dropout_rate = 0.3  # 模型丢弃率
local_epoch = 50  # 本地周期
frame_length = 16  # 补丁大小
time_step = 16  # 步长
position_device = ''  # 位置/设备
# ['chest','forearm','head','shin','thigh','upperarm','waist']
# ['nexus4', 'lgwatch','s3', 's3mini','gear','samsungold']
token_based = False  # 是否基于token

# 模型超参数
batch_size = 256
projection_dim = 192
filter_attention_head = 4
# 调整HART的块数，添加或删除卷积核大小
# 每个卷积核长度对应一个HART块
conv_kernels = [3, 7, 15, 31, 31, 31]

In [ ]:
# 输入形状和模型配置
input_shape = (segment_size, num_input_channels)
projection_half = projection_dim // 2
projection_quarter = projection_dim // 4
# 在初始化时打印这些值
print(f"projection_dim={projection_dim}, projection_half={projection_half}, projection_quarter={projection_quarter}")
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Transformer层的大小

R = projection_half // filter_attention_head
assert R * filter_attention_head == projection_half

segment_time = [x for x in range(0, segment_size - frame_length + time_step, time_step)]
assert R * filter_attention_head == projection_half
if position_device != '':
    assert dataset_name == "RealWorld" or dataset_name == "HHAR"

## 数据集和目录配置

In [ ]:
if dataset_name == 'UCI':
    ACTIVITY_LABEL = ['Walking', 'Upstair', 'Downstair', 'Sitting', 'Standing', 'Lying']
elif dataset_name == "RealWorld":
    ACTIVITY_LABEL = ['Downstairs', 'Upstairs', 'Jumping', 'Lying', 'Running', 'Sitting', 'Standing', 'Walking']
elif dataset_name == "MotionSense":
    ACTIVITY_LABEL = ['Downstairs', 'Upstairs', 'Sitting', 'Standing', 'Walking', 'Jogging']
elif dataset_name == "HHAR":
    ACTIVITY_LABEL = ['Sitting', 'Standing', 'Walking', 'Upstair', 'Downstairs', 'Biking']
else:
    # SHL
    ACTIVITY_LABEL = ['Standing', 'Walking', 'Runing', 'Biking', 'Car', 'Bus', 'Train', 'Subway']

activity_count = len(ACTIVITY_LABEL)

# 定义主目录和文件路径
main_dir = './'  # 设置主目录为当前目录

architecture_type = str(architecture) + '_' + str(int(frame_length)) + 'frameLength_' + str(time_step) + 'TimeStep_' + str(projection_dim) + "ProjectionSize_" + str(learning_rate) + 'LR'
if token_based:
    architecture_type = architecture_type + "_tokenBased"
    
if position_device != '':
    architecture_type = architecture_type + "_PositionWise_" + str(position_device)

if local_epoch < 20:
    architecture_type = "Tests/" + str(architecture_type)

# 定义文件路径
filepath = main_dir + 'HART_Results/' + architecture_type + '/' + dataset_name + '/'
os.makedirs(filepath, exist_ok=True)

attention_path = filepath + "attentionImages/"
os.makedirs(attention_path, exist_ok=True)

best_model_path = filepath + 'bestModels/'
os.makedirs(best_model_path, exist_ok=True)

current_model_path = filepath + 'currentModels/'
os.makedirs(current_model_path, exist_ok=True)
if device.type == "mps":
    print("正在使用Mac的Metal性能着色器加速")
elif device.type == "cuda":
    print(f"可用GPU数量: {torch.cuda.device_count()}")
    print(f"当前使用的GPU: {torch.cuda.get_device_name(0)}")
else:
    print("正在使用CPU")

## 加载数据集

In [ ]:
# 加载数据集
if dataset_name == "COMBINED":
    dataset_list = ["UCI", "RealWorld", "HHAR", "MotionSense", "SHL_128"]
    ACTIVITY_LABEL = ['Walk', 'Upstair', 'Downstair', 'Sit', 'Stand', 'Lay', 'Jump', 'Run', 'Bike', 'Car', 'Bus', 'Train', 'Subway']
    activity_count = len(ACTIVITY_LABEL)
    UCI = [0, 1, 2, 3, 4, 5]
    REALWORLD_CLIENT = [2, 1, 6, 5, 7, 3, 4, 0]
    HHAR = [3, 4, 0, 1, 2, 8]
    MotionSense = [2, 1, 3, 4, 0, 7]
    SHL = [4, 0, 7, 8, 9, 10, 11, 12]
    
    central_train_data = []
    central_train_label = []
    central_test_data = []
    central_test_label = []
    for dataset_name_iter in dataset_list:
        client_count = utils.return_client_by_dataset(dataset_name_iter) 
        loaded_dataset = utils.load_dataset_pytorch(dataset_name_iter, client_count, data_config, random_seed, os.path.join(main_dir, 'datasets/'))
        central_train_data.append(loaded_dataset.central_train_data.cpu().numpy())
        central_train_label.append(loaded_dataset.central_train_label.cpu().numpy())
        central_test_data.append(loaded_dataset.central_test_data.cpu().numpy())
        central_test_label.append(loaded_dataset.central_test_label.cpu().numpy())
        print(dataset_name_iter + " has class: " + str(np.unique(central_train_label[-1])))
        del loaded_dataset
    
    central_test_label_aligned = []
    central_train_label_aligned = []
    combined_aligned_data = central_test_data
    for index, dataset_name_iter in enumerate(dataset_list):
        if dataset_name_iter == 'UCI':
            central_train_label_aligned.append(central_train_label[index])
            central_test_label_aligned.append(central_test_label[index])
        elif dataset_name_iter == 'RealWorld':
            central_train_label_aligned.append(np.hstack([REALWORLD_CLIENT[label_index] for label_index in central_train_label[index]]))
            central_test_label_aligned.append(np.hstack([REALWORLD_CLIENT[label_index] for label_index in central_test_label[index]]))
        elif dataset_name_iter == 'HHAR':
            central_train_label_aligned.append(np.hstack([HHAR[label_index] for label_index in central_train_label[index]]))
            central_test_label_aligned.append(np.hstack([HHAR[label_index] for label_index in central_test_label[index]]))
        elif dataset_name_iter == 'MotionSense':
            central_train_label_aligned.append(np.hstack([MotionSense[label_index] for label_index in central_train_label[index]]))
            central_test_label_aligned.append(np.hstack([MotionSense[label_index] for label_index in central_test_label[index]]))
        else:
            central_train_label_aligned.append(np.hstack([SHL[label_index] for label_index in central_train_label[index]]))
            central_test_label_aligned.append(np.hstack([SHL[label_index] for label_index in central_test_label[index]]))
    central_train_data = np.vstack((central_train_data))
    central_test_data = np.vstack((central_test_data))
    central_train_label = np.hstack((central_train_label_aligned))
    central_test_label = np.hstack((central_test_label_aligned))
else:
    client_count = utils.return_client_by_dataset(dataset_name)
    dataset_loader = utils.load_dataset_pytorch(dataset_name, client_count, data_config, random_seed, main_dir+ 'datasets/')
    central_train_data = dataset_loader.central_train_data
    central_train_label = dataset_loader.central_train_label
    central_test_data = dataset_loader.central_test_data
    central_test_label = dataset_loader.central_test_label
    client_orientation_train = dataset_loader.client_orientation_train
    client_orientation_test = dataset_loader.client_orientation_test
    orientations_names = dataset_loader.orientations_names

In [ ]:
# 如果在RealWorld或HHAR上使用指定位置/设备，我们移除一个并将其用作测试集，并将其他用于训练
if position_device != '' or dataset_name == 'UCI':
    if dataset_name == "RealWorld":
        total_data = np.vstack((central_train_data, central_test_data))
        total_label = np.hstack((central_train_label, central_test_label))
        try:
            total_orientation = np.hstack((np.hstack((client_orientation_train)), np.hstack((client_orientation_test))))
        except:
            total_orientation = np.hstack((np.hstack([x for x in client_orientation_train]), np.hstack([x for x in client_orientation_test])))
        total_index = list(range(total_orientation.shape[0]))
        test_data_index = np.where(total_orientation == orientations_names.index(position_device))[0]
        train_data_index = np.delete(total_index, test_data_index)
        
        central_train_data = total_data[train_data_index]
        central_test_data = total_data[test_data_index]
        
        central_train_label = total_label[train_data_index]
        central_test_label = total_label[test_data_index]
    elif dataset_name == "HHAR":
        total_data = np.vstack((central_train_data, central_test_data))
        total_label = np.hstack((central_train_label, central_test_label))
        try:
            total_orientation = np.hstack((np.hstack((client_orientation_train)), np.hstack((client_orientation_test))))
        except:
            total_orientation = np.hstack((np.hstack([x for x in client_orientation_train]), np.hstack([x for x in client_orientation_test])))
        total_index = list(range(total_orientation.shape[0]))
        # 0是nexus
        test_data_index = np.where(total_orientation == orientations_names.index(position_device))[0]
        train_data_index = np.delete(total_index, test_data_index)
        
        central_train_data = total_data[train_data_index]
        central_test_data = total_data[test_data_index]
        
        central_train_label = total_label[train_data_index]
        central_test_label = total_label[test_data_index]
    # 使用位置进行评估时，没有测试集，dev=test是相同的
    central_dev_data = central_test_data
    central_dev_label = central_test_label
else:
    # 使用70 10 20比例
    central_train_data, central_dev_data, central_train_label, central_dev_label = train_test_split(
        central_train_data, central_train_label, test_size=0.125, random_state=random_seed)

In [ ]:
# 计算类权重
temp_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(central_train_label),
    y=central_train_label.ravel()
)
class_weights = {j: temp_weights[j] for j in range(len(temp_weights))}

In [ ]:
# 创建PyTorch数据集和数据加载器
# 创建特征和标签的张量
train_features = torch.FloatTensor(central_train_data)
train_labels = torch.LongTensor(central_train_label)
dev_features = torch.FloatTensor(central_dev_data)
dev_labels = torch.LongTensor(central_dev_label)
test_features = torch.FloatTensor(central_test_data)
test_labels = torch.LongTensor(central_test_label)

# 创建数据集
train_dataset = utils.HARDataset(train_features, train_labels)
dev_dataset = utils.HARDataset(dev_features, dev_labels)
test_dataset = utils.HARDataset(test_features, test_labels)

# 创建数据加载器
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True
)
dev_loader = torch.utils.data.DataLoader(
    dev_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True
)

## 创建和训练模型

In [ ]:
# 创建模型
if architecture == "HART":
    model_classifier = model.RNNAttentionHART(
        input_shape=input_shape,
        activity_count=activity_count,
        projection_dim=projection_dim,
        patch_size=frame_length,
        time_step=time_step,
        num_heads=3,
        filter_attention_head=filter_attention_head,
        conv_kernels=conv_kernels,
        dropout_rate=dropout_rate,
        use_tokens=token_based
    ).to(device)
else:
    model_classifier = model.MobileHART_XS(
        input_shape=input_shape,
        activity_count=activity_count
    ).to(device)

# 转换类权重为PyTorch张量
class_weights_tensor = torch.FloatTensor([class_weights[i] for i in range(activity_count)]).to(device)

# 创建优化器和损失函数
optimizer = torch.optim.Adam(
    model_classifier.parameters(), 
    lr=learning_rate,
    weight_decay=1e-4  # 添加适当的权重衰减
)

# 添加学习率调度器
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, 
    T_max=local_epoch,  # 总周期数
    eta_min=1e-6  # 最小学习率
)

criterion = torch.nn.CrossEntropyLoss(weight=class_weights_tensor, label_smoothing=0.1)

# 模型摘要
total_params = sum(p.numel() for p in model_classifier.parameters() if p.requires_grad)
print(f"模型总参数数量: {total_params}")

In [ ]:
# 创建检查点路径
checkpoint_filepath = filepath + "bestValcheckpoint.pt"

# 训练模型
history = {
    'train_loss': [],
    'train_accuracy': [],
    'val_loss': [],
    'val_accuracy': []
}

best_val_acc = 0.0

# 设置每个epoch的最大时间限制（秒）
max_time_per_epoch = 60  # 60秒限制

# 训练循环
start_time = time.time()
for epoch in range(local_epoch):
    # 训练阶段
    model_classifier.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    # 记录epoch开始时间
    epoch_start_time = time.time()
    time_limit_reached = False
    
    # 添加强制退出标志
    force_break = False
    
    for i, (inputs, targets) in enumerate(train_loader):
        # 严格检查时间限制 - 每批次都检查
        current_time = time.time()
        if current_time - epoch_start_time > max_time_per_epoch:
            print(f"\nEpoch {epoch+1} 已达到时间限制 ({max_time_per_epoch}秒)，提前结束！")
            print(f"已处理 {i}/{len(train_loader)} 批次 ({i/len(train_loader)*100:.1f}%)")
            time_limit_reached = True
            force_break = True
            break
            
        inputs, targets = inputs.to(device), targets.to(device)
        
        # 前向传播
        optimizer.zero_grad()
        outputs = model_classifier(inputs)
        loss = criterion(outputs, targets)
        
        # 反向传播和优化
        loss.backward()

        # 添加梯度裁剪
        torch.nn.utils.clip_grad_norm_(model_classifier.parameters(), max_norm=1.0)

        optimizer.step()
        
        # 记录损失和准确率
        train_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        train_total += targets.size(0)
        train_correct += predicted.eq(targets).sum().item()
        
        # 在每10个批次后再次检查时间（更频繁的检查）
        if i % 10 == 0 and i > 0:
            if time.time() - epoch_start_time > max_time_per_epoch:
                print(f"\nEpoch {epoch+1} 已达到时间限制 ({max_time_per_epoch}秒)，提前结束！")
                print(f"已处理 {i}/{len(train_loader)} 批次 ({i/len(train_loader)*100:.1f}%)")
                time_limit_reached = True
                force_break = True
                break
    
    # 如果时间限制导致循环退出，输出警告并确保下一步操作继续
    if force_break:
        print("警告：由于时间限制强制退出训练循环")
    
    # 只在处理了样本的情况下计算平均损失
    if train_total > 0:
        train_loss = train_loss / train_total
        train_acc = train_correct / train_total
    else:
        train_loss = float('inf')
        train_acc = 0.0
    
    # 验证阶段
    model_classifier.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for inputs, targets in dev_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            # 前向传播
            outputs = model_classifier(inputs)
            loss = criterion(outputs, targets)
            
            # 记录损失和准确率
            val_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            val_total += targets.size(0)
            val_correct += predicted.eq(targets).sum().item()
    
    val_loss = val_loss / val_total
    val_acc = val_correct / val_total
    
    # 记录历史
    history['train_loss'].append(train_loss)
    history['train_accuracy'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_accuracy'].append(val_acc)
    
    # 计算本epoch花费的时间
    epoch_time = time.time() - epoch_start_time
    
    # 打印进度
    if show_train_verbose == 1:
        current_lr = optimizer.param_groups[0]['lr']
        print(f'Epoch {epoch+1}/{local_epoch} - '
              f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, '
              f'LR: {current_lr:.6f}, Time: {epoch_time:.2f}s'
              f'{" (达到时间限制)" if time_limit_reached else ""}')
    
    # 保存最佳模型
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model_classifier.state_dict(), checkpoint_filepath)
        print(f'保存最佳模型，验证准确率: {val_acc:.4f}')

    # 更新学习率
    scheduler.step()
            
end_time = time.time() - start_time
print(f"训练时间: {end_time:.2f}秒")

# 保存当前模型
torch.save(model_classifier.state_dict(), filepath + 'bestTrain.pt')

## 模型评估与分析

In [ ]:
# 加载最佳模型
model_classifier.load_state_dict(torch.load(checkpoint_filepath))

# 测试阶段
model_classifier.eval()
test_correct = 0
test_total = 0
all_preds = []
all_targets = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        # 前向传播
        outputs = model_classifier(inputs)
        
        # 记录准确率
        _, predicted = outputs.max(1)
        test_total += targets.size(0)
        test_correct += predicted.eq(targets).sum().item()
        
        # 保存预测和真实标签
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())

# 计算测试准确率
test_acc = test_correct / test_total
print(f"测试准确率: {test_acc * 100:.2f}%")

In [ ]:
# 保存历史数据
with open(filepath + 'history.pkl', 'wb') as f:
    pickle.dump(history, f)

# 计算F1分数
weight_val_f1 = f1_score(all_targets, all_preds, average='weighted')
micro_val_f1 = f1_score(all_targets, all_preds, average='micro')
macro_val_f1 = f1_score(all_targets, all_preds, average='macro')

# 保存模型统计信息
model_statistics = {
    "Results on server model on ALL testsets": '',
    "\nTrain:": utils.round_number(max(history['train_accuracy'])),
    "\nValidation:": utils.round_number(max(history['val_accuracy'])),
    "\nTest weighted f1:": utils.round_number(weight_val_f1),
    "\nTest micro f1:": utils.round_number(micro_val_f1),
    "\nTest macro f1:": utils.round_number(macro_val_f1),
}    
with open(filepath + 'GlobalACC.csv', 'w') as f:
    w = csv.writer(f)
    w.writerows(model_statistics.items())

## 提取特征嵌入和t-SNE可视化

In [ ]:
'''# 创建一个获取中间层输出的函数
def get_intermediate_output(model, layer_name):
    # 定义前向传递钩子
    activation = {}
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output
        return hook
    
    # 找到目标层并注册钩子
    for name, module in model.named_modules():
        if "mlp_head.0" in name:  # 获取mlp_head的第一层输入，即特征提取后的表示
            module.register_forward_hook(get_activation(layer_name))
            break
    
    return activation

# 获取中间层的表示
activation_dict = get_intermediate_output(model_classifier, "features")

# 提取嵌入
embeddings = []
all_targets_list = []

with torch.no_grad():
    model_classifier.eval()
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        # 前向传播，触发钩子
        model_classifier(inputs)
        # 获取激活值
        if "features" in activation_dict:
            feature_vector = activation_dict["features"].cpu().numpy()
            embeddings.append(feature_vector)
            all_targets_list.extend(targets.numpy())

# 将所有批次的嵌入堆叠在一起
embeddings = np.vstack(embeddings)

# t-SNE可视化
perplexity = min(30.0, embeddings.shape[0] - 1)  # 确保perplexity小于样本数
tsne_model = sklearn.manifold.TSNE(perplexity=perplexity, verbose=show_train_verbose, random_state=random_seed)
tsne_projections = tsne_model.fit_transform(embeddings)

# 准备标签
labels_argmax = np.array(all_targets)
unique_labels = np.unique(labels_argmax)'''

In [ ]:
# 创建一个获取中间层输出的函数，改进版本兼容不同模型类型
def get_intermediate_output(model, layer_name):
    # 定义前向传递钩子
    activation = {}
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output
        return hook
    
    # 首先尝试检查模型是否具有extract_features方法
    if hasattr(model, 'extract_features'):
        print("模型具有extract_features方法，将使用它来获取特征")
        
        # 创建一个特殊钩子来捕获extract_features的输出
        def extract_features_hook(inputs):
            features = model.extract_features(inputs)
            activation[layer_name] = features
            return features
        
        # 保存原始方法的引用
        original_extract_features = model.extract_features
        
        # 替换为我们的钩子版本
        model.extract_features = extract_features_hook
        
        # 确保在使用后恢复原始方法
        def restore_original():
            model.extract_features = original_extract_features
        
        return activation, restore_original
    
    # 没有extract_features方法，尝试传统钩子方法
    hook_registered = False
    restore_func = lambda: None  # 默认不做任何事的恢复函数
    
    # 尝试注册到mlp_head.0（原始方法）
    for name, module in model.named_modules():
        if "mlp_head.0" in name:
            module.register_forward_hook(get_activation(layer_name))
            hook_registered = True
            print(f"钩子已注册到 {name}")
            break
    
    # 如果没有找到mlp_head.0，尝试其他常见特征提取点
    if not hook_registered:
        candidate_names = [
            "final_layer_norm",  # 常见于Transformer类模型
            "mlp_head",          # 检查整个mlp_head序列的第一层
            "transformer_layers", # Transformer最后一层
            "classifier"         # 分类层前可能有特征
        ]
        
        for candidate in candidate_names:
            if not hook_registered and hasattr(model, candidate):
                if candidate == "mlp_head" and isinstance(getattr(model, candidate), nn.Sequential):
                    # 获取第一个线性层
                    for i, module in enumerate(getattr(model, candidate)):
                        if isinstance(module, nn.Linear):
                            module.register_forward_hook(get_activation(layer_name))
                            hook_registered = True
                            print(f"钩子已注册到 mlp_head[{i}]")
                            break
                else:
                    # 直接注册到属性
                    getattr(model, candidate).register_forward_hook(get_activation(layer_name))
                    hook_registered = True
                    print(f"钩子已注册到 {candidate}")
                    break
    
    # 如果仍未找到适合的钩子点，给出警告
    if not hook_registered:
        print("警告：未找到合适的层注册钩子。特征提取可能失败。")
    
    return activation, restore_func

# 获取嵌入的增强版函数，兼容各种模型类型
def get_embeddings(model, dataloader, device):
    # 获取中间层的表示
    activation_dict, restore_func = get_intermediate_output(model_classifier, "features")
    
    # 提取嵌入
    embeddings = []
    all_targets_list = []
    
    try:
        with torch.no_grad():
            model.eval()
            for inputs, targets in dataloader:
                inputs = inputs.to(device)
                
                # 检查是否有extract_features方法并且已被我们的钩子替换
                if hasattr(model, 'extract_features') and model.extract_features.__name__ == 'extract_features_hook':
                    # 直接调用extract_features，我们的钩子会捕获结果
                    model.extract_features(inputs)
                else:
                    # 前向传播，触发常规钩子
                    model(inputs)
                
                # 获取激活值
                if "features" in activation_dict:
                    feature_vector = activation_dict["features"].cpu().numpy()
                    embeddings.append(feature_vector)
                    all_targets_list.extend(targets.numpy())
                else:
                    print("警告：此批次未捕获特征。")
        
        # 恢复原始方法（如果已修改）
        restore_func()
        
        if not embeddings:
            raise ValueError("未能提取到任何特征嵌入")
            
        # 将所有批次的嵌入堆叠在一起
        embeddings = np.vstack(embeddings)
        all_targets = np.array(all_targets_list)
        
        print(f"成功提取特征，形状: {embeddings.shape}")
        return embeddings, all_targets
    
    except Exception as e:
        # 确保恢复原始方法
        restore_func()
        
        # 尝试备用方法：检查模型是否有中间特征存储
        print(f"主要特征提取方法失败: {e}")
        print("尝试备用方法...")
        
        if hasattr(model, 'intermediate_features'):
            try:
                backup_embeddings = []
                backup_targets = []
                
                with torch.no_grad():
                    model.eval()
                    for inputs, targets in dataloader:
                        inputs = inputs.to(device)
                        _ = model(inputs)  # 运行模型
                        
                        if model.intermediate_features is not None:
                            features = model.intermediate_features.cpu().numpy()
                            backup_embeddings.append(features)
                            backup_targets.extend(targets.numpy())
                
                if backup_embeddings:
                    embeddings = np.vstack(backup_embeddings)
                    all_targets = np.array(backup_targets)
                    print(f"备用方法成功，提取特征形状: {embeddings.shape}")
                    return embeddings, all_targets
            except Exception as backup_e:
                print(f"备用方法也失败了: {backup_e}")
        
        # 如果所有方法都失败，重新抛出原始异常
        raise ValueError(f"所有特征提取方法均失败: {e}")

# 使用增强的特征提取函数获取嵌入
try:
    print("开始提取特征嵌入...")
    embeddings, all_targets_array = get_embeddings(model_classifier, test_loader, device)

    # t-SNE可视化
    print("开始t-SNE降维...")
    perplexity = min(30.0, embeddings.shape[0] - 1)  # 确保perplexity小于样本数
    tsne_model = sklearn.manifold.TSNE(perplexity=perplexity, verbose=show_train_verbose, random_state=random_seed)
    tsne_projections = tsne_model.fit_transform(embeddings)

    # 准备标签
    labels_argmax = all_targets_array
    unique_labels = np.unique(labels_argmax)

    print(f"t-SNE降维完成，投影形状: {tsne_projections.shape}")
    print(f"识别到的活动类别: {len(unique_labels)}")

    # 绘制t-SNE图
    if (dataset_name == 'RealWorld' or dataset_name == 'HHAR') and position_device == '':
        utils.project_tsne_with_position(dataset_name, architecture + "_TSNE_Embeds", filepath, ACTIVITY_LABEL,
                                       labels_argmax, orientations_names, client_orientation_test,
                                       tsne_projections, unique_labels)
    else:
        utils.project_tsne(architecture + "_TSNE_Embeds", filepath, ACTIVITY_LABEL,
                         labels_argmax, tsne_projections, unique_labels)
    
except Exception as e:
    print(f"t-SNE可视化过程失败: {e}")
    
    # 提供简单的备用可视化函数
    def simple_tsne_plot(save_path):
        """简单的特征可视化函数，不依赖于特征提取"""
        print("使用简单备用可视化...")
        
        # 收集最后线性层之前的特征
        backup_features = []
        backup_labels = []
        
        # 尝试获取分类器前的特征
        with torch.no_grad():
            model_classifier.eval()
            
            # 找到最后一个线性层之前的层
            pre_final_layer = None
            for name, module in model_classifier.named_modules():
                if isinstance(module, nn.Linear) and 'classifier' in name:
                    # 找到了分类器，现在找它的输入
                    for parent_name, parent_module in model_classifier.named_modules():
                        if parent_name and parent_name in name and parent_name != name:
                            pre_final_layer = parent_module
                            break
                    break
            
            if pre_final_layer is None:
                print("警告：无法识别预分类层，将使用随机数据进行示例可视化")
                # 生成随机数据进行演示
                random_features = np.random.rand(100, 2)
                random_labels = np.random.randint(0, len(ACTIVITY_LABEL), 100)
                
                plt.figure(figsize=(10, 10))
                for i in range(len(ACTIVITY_LABEL)):
                    mask = random_labels == i
                    if np.any(mask):
                        plt.scatter(random_features[mask, 0], random_features[mask, 1], 
                                   label=ACTIVITY_LABEL[i])
                
                plt.legend()
                plt.title("示例t-SNE可视化（使用随机数据）")
                plt.savefig(save_path, dpi=300)
                plt.close()
                return
            
            # 使用钩子捕获特征
            activation = {}
            def hook(module, input, output):
                activation['features'] = input[0]
            
            hook_handle = pre_final_layer.register_forward_hook(hook)
            
            for inputs, targets in test_loader:
                inputs = inputs.to(device)
                _ = model_classifier(inputs)
                
                if 'features' in activation:
                    features = activation['features'].cpu().numpy()
                    backup_features.append(features)
                    backup_labels.extend(targets.numpy())
            
            hook_handle.remove()
            
            if backup_features:
                # 创建t-SNE可视化
                backup_features = np.vstack(backup_features)
                backup_labels = np.array(backup_labels)
                
                # t-SNE降维
                perplexity = min(30.0, backup_features.shape[0] - 1)
                backup_tsne = sklearn.manifold.TSNE(
                    perplexity=perplexity, 
                    random_state=random_seed
                ).fit_transform(backup_features)
                
                # 创建可视化
                plt.figure(figsize=(16, 16))
                for i in range(len(ACTIVITY_LABEL)):
                    mask = backup_labels == i
                    if np.any(mask):
                        plt.scatter(
                            backup_tsne[mask, 0], 
                            backup_tsne[mask, 1],
                            label=ACTIVITY_LABEL[i]
                        )
                
                plt.legend(fontsize=12)
                plt.title('t-SNE特征可视化（备用方法）', fontsize=16)
                plt.savefig(save_path, dpi=300, bbox_inches='tight')
                plt.close()
                print(f"备用t-SNE图已保存至: {save_path}")
    
    # 如果主要可视化失败，使用简单备用方法
    simple_tsne_plot(filepath + architecture + "_TSNE_Embeds_backup.png")

In [ ]:
'''# 绘制t-SNE图
if (dataset_name == 'RealWorld' or dataset_name == 'HHAR') and position_device == '':
    utils.project_tsne_with_position(dataset_name, architecture + "_TSNE_Embeds", filepath, ACTIVITY_LABEL,
                                   labels_argmax, orientations_names, client_orientation_test,
                                   tsne_projections, unique_labels)
else:
    utils.project_tsne(architecture + "_TSNE_Embeds", filepath, ACTIVITY_LABEL,
                     labels_argmax, tsne_projections, unique_labels)'''

## 绘制混淆矩阵和注意力可视化

In [ ]:
# 绘制混淆矩阵
results = confusion_matrix(all_targets, all_preds)
df_cm = pd.DataFrame(results, index=[i for i in ACTIVITY_LABEL],
                  columns=[i for i in ACTIVITY_LABEL])
plt.figure(figsize=(14, 14))
sns.set(font_scale=1.4)
sns.heatmap(df_cm, annot=True, cmap=plt.cm.Blues, cbar=False)
plt.ylabel('预测')
plt.xlabel('真实')
plt.savefig(filepath + 'HeatMap.png')
plt.show()

In [ ]:
# 获取模型中的关键层
if architecture == "HART":
    final_acc_mha_name = get_layer_index_by_name(model_classifier, f"AccMHA_{len(conv_kernels)-1}")
    final_gyro_mha_name = get_layer_index_by_name(model_classifier, f"GyroMHA_{len(conv_kernels)-1}")
    final_inputs_name = get_layer_index_by_name(model_classifier, f"normalizedInputs_{len(conv_kernels)-1}")
    
    # 创建hook函数获取中间输出
    intermediate_outputs = {}
    def get_activation(name):
        def hook(model, input, output):
            intermediate_outputs[name] = output
        return hook
    
    # 注册钩子
    for name, module in model_classifier.named_modules():
        if name == final_inputs_name:
            module.register_forward_hook(get_activation(final_inputs_name))
            
    # 选择一些样本用于可视化
    indices = []
    for label in range(activity_count):
        label_indices = np.where(np.array(all_targets) == label)[0]
        if len(label_indices) > 0:
            indices.append(label_indices[2] if len(label_indices) > 2 else label_indices[0])
    
    segment_time = [x for x in range(0, segment_size - frame_length + time_step, time_step)]
    
    # 可视化注意力
    for index, class_loc in enumerate(indices):
        # 获取样本
        sample = test_features[class_loc].unsqueeze(0).to(device)
        
        # 清空中间输出
        intermediate_outputs.clear()
        
        # 前向传播
        with torch.no_grad():
            model_classifier(sample)
            
        # 注意：这里我们模拟注意力可视化
        # 实际上需要从模型中提取注意力权重
        # 由于PyTorch版本的模型结构不同，这里仅为示例
        
        # 生成伪注意力分数
        attention_scores = np.random.rand(len(segment_time))
        attention_scores_norm = ((attention_scores - min(attention_scores))/(max(attention_scores) - min(attention_scores)) -1) * - 0.5
        
        # 绘制第一组图（加速度计数据）
        gs = gridspec.GridSpec(2, 1)
        fig = plt.figure(figsize=(10, 8))
        plt.title(f"Attention Map for {ACTIVITY_LABEL[index]}", size=16)
        plt.margins(x=0)
        plt.tick_params(
        axis='both',
        which='both',
        labelleft=False,
        left=False,
        bottom=False,
        top=False,
        labelbottom=False)
        
        # 加速度计数据
        ax = fig.add_subplot(gs[0])
        ax.margins(x=0)
        
        sample_np = sample.cpu().numpy()[0]
        ax.plot(sample_np[:, 0], label="x-axis")
        ax.plot(sample_np[:, 1], label="y-axis")
        ax.plot(sample_np[:, 2], label="z-axis")
        
        for bar_index, start_time in enumerate(segment_time):
            ax.axvspan(start_time, start_time + frame_length, facecolor='black', alpha=float(attention_scores_norm[bar_index]), zorder=4)
        
        ax.set_ylabel(r'Acc ($m/s^2$)', size=16)
        ax.get_yaxis().set_label_coords(-0.1, 0.5)
        ax.tick_params(axis='x', labelbottom=False)
        plt.legend(loc='upper right', framealpha=0.7)
        
        # 生成第二组伪注意力分数
        attention_scores = np.random.rand(len(segment_time))
        attention_scores_norm = ((attention_scores - min(attention_scores))/(max(attention_scores) - min(attention_scores)) -1) * - 0.5
        
        # 陀螺仪数据
        ax = fig.add_subplot(gs[1], sharex=ax)
        ax.margins(x=0)
        
        ax.plot(sample_np[:, 3], label="x-axis")
        ax.plot(sample_np[:, 4], label="y-axis")
        ax.plot(sample_np[:, 5], label="z-axis")
        
        for bar_index, start_time in enumerate(segment_time):
            ax.axvspan(start_time, start_time + frame_length, facecolor='black', alpha=float(attention_scores_norm[bar_index]), zorder=99)
        
        ax.set_ylabel(r'Gyro (rad/s)', size=16)
        ax.get_yaxis().set_label_coords(-0.1, 0.5)
        plt.xticks([0, 32, 64, 96, 128])
        if len(fig.get_axes()) > 1:
            fig.get_axes()[1].set_xticklabels([0, 0.64, 1.28, 1.9, 2.56])
        plt.xlabel("Time (s)", size=16)
        plt.margins(x=0)
        
        plt.savefig(attention_path + ACTIVITY_LABEL[index] + "MeanHeadAttention.png", bbox_inches="tight")
        plt.show()
        plt.close()

## 学习曲线和模型导出

In [ ]:
# 绘制学习曲线
utils.plot_learning_curve(history, local_epoch, filepath)

In [ ]:
# 导出ONNX模型
# TODO:模型导出有bug
'''dummy_input = torch.randn(1, segment_size, num_input_channels, device=device)
torch.onnx.export(
    model_classifier,
    dummy_input,
    filepath + architecture + '.onnx',
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
)

print("模型已导出为ONNX格式")'''

In [ ]:
# 显示模型性能统计
print(f"训练准确率: {max(history['train_accuracy']) * 100:.2f}%")
print(f"验证准确率: {max(history['val_accuracy']) * 100:.2f}%")
print(f"测试准确率: {test_acc * 100:.2f}%")
print(f"测试加权F1分数: {weight_val_f1:.4f}")
print(f"测试微平均F1分数: {micro_val_f1:.4f}")
print(f"测试宏平均F1分数: {macro_val_f1:.4f}")
print("训练完成!")